<a href="https://colab.research.google.com/github/inasalkamachy/Introduction-to-TensorFlow-for-Artificial-Intelligence-Machine-Learning-and-Deep-Learning-Week-2/blob/main/DogVsCat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import os
import zipfile
from zipfile import ZipFile

zip_ref = zipfile.ZipFile("DVC.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [30]:

# Directory with our training horse pictures
train_cat_dir = os.path.join('DVC/cat')

# Directory with our training human pictures
train_dog_dir = os.path.join('DVC/dog')

In [31]:
train_cat_names = os.listdir(train_cat_dir)
train_dog_names = os.listdir(train_dog_dir)

In [32]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy')>0.999):
                print("/n Reached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True

callbacks = myCallback()

In [33]:
import tensorflow as tf
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])


from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [34]:

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 33, 33, 64)       

In [35]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [36]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'DVC',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=4,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 30 images belonging to 2 classes.


In [38]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 4s 362ms/step - loss: 3.1919 - accuracy: 0.4115
Epoch 2/15
8/8 [==============================] - 3s 371ms/step - loss: 0.6130 - accuracy: 0.7030
Epoch 3/15
8/8 [==============================] - 3s 379ms/step - loss: 0.7015 - accuracy: 0.6574
Epoch 4/15
8/8 [==============================] - 3s 343ms/step - loss: 0.7193 - accuracy: 0.4729
Epoch 5/15
8/8 [==============================] - 3s 342ms/step - loss: 0.7267 - accuracy: 0.5513
Epoch 6/15
8/8 [==============================] - 3s 342ms/step - loss: 0.6781 - accuracy: 0.5722
Epoch 7/15
8/8 [==============================] - 3s 358ms/step - loss: 0.6910 - accuracy: 0.6363
Epoch 8/15
8/8 [==============================] - 3s 366ms/step - loss: 1.0833 - accuracy: 0.4859
Epoch 9/15
8/8 [==============================] - 3s 346ms/step - loss: 0.6659 - accuracy: 0.6063
Epoch 10/15
8/8 [==============================] - 3s 353ms/step - loss: 0.6872 - accuracy: 0.8482
Epoch 11/15
8/8 [==

In [41]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

Saving 11885.jpg to 11885.jpg
[0.9999826]
11885.jpg is a dog
